In [1]:
from preprocessing.preprocessing import  get_preprocessed_brfss_train_test_split, diabetes_columns
from sklearn.tree import DecisionTreeClassifier
import graphviz
from sklearn import tree
from IPython.core.display_functions import display

   GenHealth  PhysHealth  MentHealth  Healthcare  MedCost  Checkup  HighBP  \
0        5.0        15.0        18.0         1.0      2.0      1.0     1.0   
1        3.0        88.0        88.0         2.0      1.0      4.0     3.0   
2        4.0        15.0        88.0         1.0      2.0      1.0     3.0   
3        5.0        30.0        30.0         1.0      1.0      1.0     1.0   
4        5.0        20.0        88.0         1.0      2.0      1.0     3.0   

   HighChol  HeartAttack  AngiCoro  Stroke  Asthma  Arthritis  Kidney  Sex  \
0       1.0          2.0       2.0     2.0     1.0        1.0     2.0  2.0   
1       2.0          2.0       2.0     2.0     2.0        2.0     2.0  2.0   
2       1.0          7.0       2.0     1.0     2.0        1.0     2.0  2.0   
3       1.0          2.0       2.0     2.0     2.0        1.0     2.0  2.0   
4       2.0          2.0       2.0     2.0     2.0        1.0     2.0  2.0   

   Income  SodiumSalt   Age  Height   Weight  BMI  Education  

In [2]:
data_train, data_test, target_train, target_test = get_preprocessed_brfss_train_test_split()
data_train.head(10)

,GenHealth,PhysHealth,MentHealth,Healthcare,MedCost,Checkup,HighBP,HighChol,HeartAttack,AngiCoro,Stroke,Asthma,Arthritis,Kidney,Sex,Income,SodiumSalt,Age,Height,Weight,BMI,Education,Alcohol,Smoking,FruitCons,VegetCons,PhysActivity,Muscles
434107,2.0,88.0,88.0,1.0,2.0,3.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,8.0,0.0,9.0,152.0,5897.0,3.0,2.0,2.0,4.0,2.0,1.0,4.0,2.0
110299,4.0,88.0,88.0,1.0,2.0,1.0,3.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,1.0,4.0,2.0,13.0,175.0,7484.0,2.0,2.0,1.0,4.0,1.0,2.0,3.0,2.0
218193,3.0,88.0,88.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,77.0,0.0,8.0,180.0,8165.0,3.0,2.0,2.0,3.0,1.0,1.0,1.0,2.0
394642,2.0,88.0,1.0,1.0,1.0,2.0,3.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,5.0,0.0,1.0,165.0,5443.0,2.0,2.0,1.0,4.0,1.0,1.0,1.0,2.0
354955,1.0,1.0,88.0,1.0,2.0,3.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,7.0,1.0,7.0,165.0,9480.0,4.0,2.0,1.0,3.0,1.0,1.0,1.0,1.0
228337,1.0,88.0,88.0,1.0,2.0,1.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,6.0,0.0,3.0,168.0,8391.0,0.0,4.0,1.0,4.0,1.0,1.0,4.0,2.0
57536,2.0,88.0,3.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,8.0,0.0,9.0,165.0,6350.0,2.0,4.0,1.0,4.0,2.0,1.0,2.0,2.0
68009,3.0,88.0,88.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,7.0,0.0,11.0,165.0,6577.0,2.0,4.0,2.0,3.0,1.0,1.0,2.0,1.0
308419,3.0,88.0,88.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,6.0,0.0,11.0,155.0,6577.0,3.0,3.0,1.0,3.0,1.0,1.0,2.0,2.0
359773,1.0,5.0,3.0,1.0,2.0,1.0,1.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,5.0,0.0,1.0,175.0,23088.0,4.0,2.0,2.0,3.0,1.0,1.0,3.0,1.0


In [3]:
decision_tree_classifier = DecisionTreeClassifier()
decision_tree_classifier.fit(data_train, target_train)

DecisionTreeClassifier()

In [ ]:
dot_data = tree.export_graphviz(decision_tree_classifier, out_file=None,
                                feature_names=data_train.columns,
                                class_names=diabetes_columns,
                                filled=True, rounded=True, special_characters=True)

graph = graphviz.Source(dot_data)

In [6]:
from sklearn.model_selection import cross_val_score, StratifiedKFold

cross_validation = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
accuracy = cross_val_score(decision_tree_classifier, data_train, target_train, cv=cross_validation, scoring='accuracy')

for i, acc in enumerate(accuracy):
    print("Fold {}: Accuracy = {}%".format(i, acc * 100.0))

print("Average Accuracy = {}%".format(accuracy.mean() * 100.0))

Fold 0: Accuracy = 75.89477857061955%
Fold 1: Accuracy = 75.97689432551817%
Fold 2: Accuracy = 75.85230490429267%
Fold 3: Accuracy = 76.50639936572658%
Fold 4: Accuracy = 75.80699966021068%
Fold 5: Accuracy = 76.05051534715143%
Fold 6: Accuracy = 75.7645259938838%
Fold 7: Accuracy = 76.31385207837808%
Fold 8: Accuracy = 76.12979952429494%
Fold 9: Accuracy = 75.85728444003963%
Average Accuracy = 76.01533542101156%


In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import pandas as pd

decision_tree_classifier = DecisionTreeClassifier()
# specify the parameter grid
parameters = {
    'max_depth': range(5, 28),
    'splitter': ["best", "random"],
    'criterion': ["gini", "entropy", "log_loss"]
}


# create the grid search instance
grid_search_estimator = GridSearchCV(decision_tree_classifier, parameters, scoring='accuracy', cv=cross_validation, return_train_score=False)

# run the grid search
grid_search_estimator.fit(data_train, target_train)

# print the results of all hyper-parameter combinations
results = pd.DataFrame(grid_search_estimator.cv_results_)
display(results)

# print the best parameter setting
print("best score is {} with params {}".format(grid_search_estimator.best_score_, grid_search_estimator.best_params_))

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.472740,0.013230,0.004462,0.000285,gini,5,best,"{'criterion': 'gini', 'max_depth': 5, 'splitte...",0.846982,0.846925,0.846698,0.846812,0.845934,0.846982,0.846557,0.846840,0.846415,0.846043,0.846619,0.000360,13
1,0.205233,0.003864,0.004170,0.000216,gini,5,random,"{'criterion': 'gini', 'max_depth': 5, 'splitte...",0.843782,0.842932,0.845679,0.842961,0.843300,0.843131,0.841941,0.842904,0.843074,0.842673,0.843238,0.000926,45
2,0.540735,0.004124,0.004417,0.000180,gini,6,best,"{'criterion': 'gini', 'max_depth': 6, 'splitte...",0.848511,0.847180,0.846670,0.846840,0.846387,0.847095,0.846642,0.847208,0.847208,0.845986,0.846973,0.000638,6
3,0.240472,0.004426,0.004292,0.000166,gini,6,random,"{'criterion': 'gini', 'max_depth': 6, 'splitte...",0.842847,0.842904,0.846415,0.846217,0.842649,0.846415,0.845113,0.846047,0.846528,0.843013,0.844815,0.001646,36
4,0.619247,0.001294,0.004644,0.000161,gini,7,best,"{'criterion': 'gini', 'max_depth': 7, 'splitte...",0.848397,0.847406,0.846557,0.847435,0.846783,0.847180,0.847038,0.848001,0.847378,0.846411,0.847259,0.000584,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,0.757129,0.007152,0.010688,0.000204,log_loss,25,random,"{'criterion': 'log_loss', 'max_depth': 25, 'sp...",0.783639,0.786839,0.782507,0.780298,0.782818,0.781572,0.782365,0.783214,0.787377,0.785813,0.783644,0.002194,119
134,1.590334,0.004536,0.010550,0.000057,log_loss,26,best,"{'criterion': 'log_loss', 'max_depth': 26, 'sp...",0.771916,0.770869,0.771605,0.771633,0.768887,0.774040,0.772879,0.773304,0.777296,0.770409,0.772284,0.002184,136
135,0.767439,0.005688,0.010924,0.000242,log_loss,26,random,"{'criterion': 'log_loss', 'max_depth': 26, 'sp...",0.776277,0.780779,0.779165,0.778061,0.776900,0.779420,0.777919,0.781799,0.776589,0.778196,0.778511,0.001707,126
136,1.598364,0.004149,0.010420,0.000092,log_loss,27,best,"{'criterion': 'log_loss', 'max_depth': 27, 'sp...",0.771095,0.768632,0.769340,0.769906,0.768207,0.772624,0.771492,0.771237,0.775626,0.767889,0.770605,0.002232,138


best score is 0.847258597833485 with params {'criterion': 'gini', 'max_depth': 7, 'splitter': 'best'}
